# Chapitre II: Notions générales

## Sommaire
* [Modélisation statistique du language](#modelisation-statistique-du-language)
* [Bag of Words (BoW)](#bag-of-words-bow)
   * [Implémentation en Python:](#implementation-en-python)
* [Term Frequency-Inverse Document Frenquency (TF-IDF)](#term-frequency-inverse-document-frenquency-tf-idf)
   * [Implémentation en Python:](#id1)
   
## Modélisation statistique du language

Maintenant que nous avons vu comment préparer le texte avant son utilisation dans un algorithme de NLP, nous nous intéresserons à la représentation des phrases d'un point de vue statistique.

Il est difficile pour un algorithme de travailler avec du texte, mais si nous pouvons convertir ce texte en une suite de chiffres alors nous pourrons en tirer certaines informations qui permettront à l'algorithme de travailler.

Nous étudierons deux techniques de modélisation du texte sous forme de vecteur:
* Bag of Words (BoW)
* Term Frequency-Inverse Document Frequency (TF-IDF)

### Bag of Words (BoW)

Bag of Words ou Sac de Mots est une technique de modélisation du texte sous forme de vecteur où chaque élément représente la fréquence d'un token (ici mots) dans le corpus.

Par exemple:

```
John aime regarder des films, Mary aime les films aussi.
```

Deviendra, après tokenisation et regroupement des mots:

```
"John", "aime", "regarder", "des", "films", "Mary", "les", "aussi"
```

On obtiendra alors la répresentation vectorielle suivante:

```
[1, 2, 1, 1, 2, 1, 1, 1]
```

*Ce vecteur ne préserve pas l'ordre des mots.*

#### Implémentation en Python:

Il est possible de le programmer soi-même ou d'utiliser une librairie Python tel que scikit-learn qui fera aussi le pré-processing:

``` python
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

sentence = "John aime regarder des films, Mary aime les films aussi."

CountVec = CountVectorizer(ngram_range=(1,1), stop_words=stopwords.words('french'))

Count_data = CountVec.fit_transform([sentence])

print(CountVec.get_feature_names_out())
print(Count_data.toarray())
```

On obtient alors:

```
['aime' 'aussi' 'films' 'john' 'mary' 'regarder']
[[2 1 2 1 1 1]]
```

Cependant nous pouvons nous poser la question de la pertinence de cette information: un mot fréquent est il porteur de sens ?

La réponse est non, si un mot survient plusieurs fois dans un document mais aussi dans un nombre important de documents alors, peut être, que ce mot est juste un mot fréquent comme une conjonction de coordination, sans sens particulier donc.

### Term Frequency-Inverse Document Frenquency (TF-IDF)

Afin de remédier au problème du Sac de Mots une approche serait de redéfinir la fréquence des mots afin de constituer un indice qui permette de connaître l'importance d'un mot dans un corpus de textes, ainsi les mots présent dans de nombreux documents seront pénalisés.

Pour cela on utilise deux indices que l'on multipliera:
1. Term Frequency
2. Inverse Document Frequency

Tout d'abord le Term Frequency ou la fréquence des termes se calcule de la manière suivante:

$$TF = \frac{n}{N} \qquad \text{Où:} \begin{equation} \begin{cases} n = \text{Nombre d'instance du mot dans le document} \\ N = \text{Nombre total de mots dans le document} \end{cases} \end{equation}$$

Ensuite nous avons l'Inverse Document Frequency ou fréquence inverse du document qui est calculée en prenant le nombre total de documents du dataset, le divisant par le nombre de document contenant le mot puis en appliquant un logarithme:

$$IDF = 1+\log(\frac{D}{Dn}) \qquad \text{Où:} \begin{equation} \begin{cases} D = \text{Nombre total de documents} \\ Dn = \text{Nombre total de documents contenant le mot} \end{cases} \end{equation}$$

Enfin nous multiplions ces deux termes pour obtenir le Term frequency-Inverse Document Frequency (TF-IDF):

$$TF-IDF = TF*IDF \Rightarrow TF-IDF = \frac{n}{N}+\frac{n}{N}log(\frac{D}{Dn})$$

#### Implémentation en Python:

``` python
import re
import nltk
import unidecode
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

#nltk.download('wordnet')

def text_processing(text):
    ''' Return cleaned text for Machine Learning '''
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    NEW_LINE = re.compile('\n')
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    STEMMER = SnowballStemmer('french')

    text = text.lower()
    text = unidecode.unidecode(text)
    text = NEW_LINE.sub(' ',text)
    text = REPLACE_BY_SPACE_RE.sub(' ',text)
    text = BAD_SYMBOLS_RE.sub(' ',text)
    text = ' '.join([STEMMER.stem(word) for word in word_tokenize(text)])
    return text

sentence1 = "John aime regarder des films, Mary aime les films aussi."
sentence2 = "Bertrand aime les pizzas, mais les pizzas ne l'aiment pas en retour."
sentence3 = "Le muay thai est supérieur au kick boxing qui est lui même supérieur à la boxe anglaise"

tf_idf_vec = TfidfVectorizer(use_idf=True, ngram_range=(1,1), stop_words=stopwords.words('french'))

tf_idf_data = tf_idf_vec.fit_transform([text_processing(sentence1),text_processing(sentence2), text_processing(sentence3)])

print(tf_idf_vec.get_feature_names_out())
print(tf_idf_data.toarray())
```

on obtient alors:

```
['aim' 'aiment' 'anglais' 'auss' 'bertrand' 'box' 'boxing' 'film' 'john'
 'kick' 'mary' 'mem' 'muay' 'pizz' 'regard' 'retour' 'superieur' 'thai']
[[0.4736296  0.         0.         0.311383   0.         0.
  0.         0.62276601 0.311383   0.         0.311383   0.
  0.         0.         0.311383   0.         0.         0.        ]
 [0.27626457 0.36325471 0.         0.         0.36325471 0.
  0.         0.         0.         0.         0.         0.
  0.         0.72650942 0.         0.36325471 0.         0.        ]
 [0.         0.         0.30151134 0.         0.         0.30151134
  0.30151134 0.         0.         0.30151134 0.         0.30151134
  0.30151134 0.         0.         0.         0.60302269 0.30151134]]
```

*Malheureusement le Stemmer a commis quelques erreurs par exemple 'aim' et 'aiment' ou 'box' et 'boxing' qui pourtant devraient avoir la même racine.*